In [ ]:
# IN questo notebook vado a leggere il file delle email (quello contenente tutte le email ricevute ed inviate da ogni persona) e vado a calcolare le prime feature per ogni utente.
#ATTENZIONE: non abbiamo ancor auna network, qui calcoliamo solo le feature inerenti alle email invate e ricevute ed ai vari tipi di destinatari

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.ticker import MaxNLocator

csv_file_path = '/content/gdrive/MyDrive/processati_email/chunked_userID'


mapping1 = {
        'EBDCFA':'BE',
        'EBDPBK':'BE',
        'EBDRPF':'BE',
        'EUROBA':'BE',

        'EUROFO':'SGR',
        'SGRCCE':'SGR',
        'SGRDBL':'SGR',
        'SGRINV':'SGR',

        'PRIVAT':'CENTRO PRIVATE CREDEM',
        'COPRIV':'CENTRO PRIVATE CREDEM',

        'RP01':'DRC',

        'CSDIR': 'CDMASS',

        'CVDIR': 'CDMVIT',

        'VDGPM':'STAFF DIREZIONE',
        'DIRGEN':'STAFF DIREZIONE',

        'VDGBC':'COMME'
        }
df_anagrafici_excel = pd.read_excel(io = "/content/gdrive/MyDrive/Colab Notebooks/Dati per POLIMI RIVISTO_OUTPUT.xlsx", engine='openpyxl', sheet_name= 'OUTPUT')
df_anagrafici_excel['UO sup'] = df_anagrafici_excel['UO sup'].replace(mapping1)


id_email_unique=[]
for filename in os.listdir(csv_file_path):
    if filename.endswith(".csv"):

        # Extract the user_id from the filename by removing the ".csv" extension
        ID_persona = filename[:-4]
        id_email_unique.append(ID_persona)


## This dataframe DOESN'T CONTAIN OUTLIERS (according to Gabriele Boccoli)
df_survey = pd.read_excel(io = "/content/gdrive/MyDrive/Colab Notebooks/JRC_Credem_Survey.xlsx", engine='openpyxl', sheet_name= 'Dataset_nooutlier') #1495 utenti

column_data_types = {
    'CODICI RISPONDENTI': str,
    'SCALA 100': float
}
df_retribuzione_excel = pd.read_excel(io = "/content/gdrive/MyDrive/Colab Notebooks/Dataset_retribuzione scala 100_OUTPUT.xlsx", engine='openpyxl', sheet_name='PANEL RETR SCALA 100 OUTPUT', dtype=column_data_types)



print("persone in anagragica: ", df_anagrafici_excel['CODICI RISPONDENTI'].nunique() )
print("persone nella survey: ", df_survey['PASSWORD'].nunique() )
print("persone in retribuzione: ", df_retribuzione_excel['CODICI RISPONDENTI'].nunique() )
print("persone nelle email: ", len(id_email_unique) )




persone in anagragica:  2041
persone nella survey:  1495
persone in retribuzione:  2041
persone nelle email:  2041


In [ ]:
print(df_anagrafici_excel['UO sup'].nunique())

print(df_anagrafici_excel['UO sup'].unique())

50
['SINFO' 'DT95' 'CESER' 'CREDI' 'DT80' 'PEOPLE' 'DRC' 'DT11' 'DT71'
 'CENTRO PRIVATE CREDEM' 'COMME' 'RPB' 'DT90' 'DT12' 'LEADIR' 'ANTRIC'
 'FCTDIR' 'DT20' 'DT70' 'CDMVIT' 'DT40' 'FINANZ' 'DT50' 'DT30' 'DT13'
 'LOGIS' 'DT10' 'CDMTEL' 'AMMIN' 'ERISK' 'BE' 'STAFF DIREZIONE' 'CDMASS'
 'AUDIT' 'VALORE' 'DT25' 'LEGAL' 'ORGAN' 'GOVREL' 'STAFDC' 'TUTELA' 'SGR'
 'AVVERA' 'SIMDIR' 'COMPLY' 'DT72' 'EUROFI' 'COWEMA' 'BESDR' 'SATADR']


In [ ]:

df_features=pd.DataFrame()

#inizializzo il dataframe con gli ID
df_features['user_ID'] = id_email_unique

df_features['addettoVSmanager'] = None
df_features['performance'] = 0 #None
df_features['seniority'] = 0
df_features['retribuzione'] = 0.0
df_features['BU_sup'] = None
df_features['BU_inf'] = None

#print( len(df_retribuzione_excel['CODICI RISPONDENTI'].unique()))

df_features['total_sent_email'] = 0
df_features['total_received_email'] = 0
df_features['#persone_interazioni'] = 0

df_features['sent_to_EXTERNAL'] = 0
df_features['received_from_EXTERNAL'] = 0

df_features['sent_to_CONSULENTI'] = 0
df_features['received_from_CONSULENTI'] = 0

df_features['people_sent_SAME_bu_SUP'] = 0
df_features['email_sent_SAME_bu_SUP'] = 0
df_features['people_sent_SAME_bu_INF'] = 0
df_features['email_sent_SAME_bu_INF'] = 0

df_features['email_sent_different_bu_SUP'] = 0
df_features['people_sent_different_bu_SUP'] = 0
df_features['people_sent_different_bu_INF'] = 0
df_features['email_sent_different_bu_INF'] = 0

df_features['people_received_SAME_bu_SUP'] = 0
df_features['email_received_SAME_bu_SUP'] = 0
df_features['people_received_SAME_bu_INF'] = 0
df_features['email_received_SAME_bu_INF'] = 0

df_features['people_received_DIFFERENT_bu_SUP'] = 0
df_features['email_received_DIFFERENT_bu_SUP'] = 0
df_features['email_received_DIFFERENT_bu_INF'] = 0
df_features['people_received_DIFFERENT_bu_INF'] = 0


df_features['total_SAME_bu_SUP'] = 0
df_features['people_total_SAME_bu_SUP'] = 0

df_features['total_SAME_bu_INF'] = 0
df_features['people_total_SAME_bu_INF'] = 0

df_features['total_DIFFERENT_bu_SUP'] = 0
df_features['people_total_DIFFERENT_bu_SUP'] = 0

df_features['total_DIFFERENT_bu_INF'] = 0
df_features['people_total_DIFFERENT_bu_INF'] = 0


df_features['persone_sent_to_ADDETTO'] = 0
df_features['email_sent_to_ADDETTO'] = 0

df_features['persone_sent_to_MANAGER'] = 0
df_features['email_sent_to_MANAGER'] = 0

df_features['persone_received_from_ADDETTO'] = 0
df_features['email_received_from_ADDETTO'] = 0

df_features['persone_received_from_MANAGER'] = 0
df_features['email_received_from_MANAGER'] = 0

# Definisci il mapping dei valori testuali
mapping = {
        'ECCELLENTI': 7,
        'RILEVANTI': 6,
        'APPREZZABILI': 5,
        'SODDISFACENTI': 4,
        'ADEGUATI': 3,
        'CARENTI': 2,
        'INADEGUATI': 1
        }

## Il campo companySeniority viene calcolato come differenza dell'anno in cui siamo oggi e l'anno in cui è stato assunto: "Ultimo colloquio"
from datetime import date
today = date.today()
currentYear = today.strftime("%Y")


In [ ]:
# Siccome non l'ho fatto il precedenza, vado ad eliminare tutte quelle email che avevano un numero di destinatari eccessivamente alto!
# metto la soglia a 25 destinatari!


# Install pyspark
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# 1) Create a SparkSession
spark = SparkSession.builder.appName("CSVtoCHUNK").config("spark.driver.memory", "16g").getOrCreate()

# 2) spark dataframe
df_email_spark = None

folder_path = '/content/gdrive/MyDrive/processati_email'

i=0

for filename in os.listdir(folder_path):

    if filename.startswith('email_') and filename.endswith('.csv'):
        i=i+1
        file_path = os.path.join(folder_path, filename)

        #if i ==2: break

        #read the CSV files into temporary Spark DataFrames (temp_df),
        # and append them to df_email_spark using the union method
        temp_df = spark.read.csv(file_path, header=True, inferSchema=True)

        if df_email_spark is None:
            df_email_spark = temp_df
            print(i , ' FILE caricato:', filename)# , ', righe:', df_email_spark.count())
        else:
            df_email_spark = df_email_spark.union(temp_df)
            print(i , ' FILE caricato:', filename)# , ':', df_email_spark.count())


grouped = df_email_spark.groupBy("message_id").agg(
    F.collect_list("destination_address").alias("lista_destinatari")
)

# Create a DataFrame with columns "message_id" and "lista_destinatari"
result_df_dest = grouped.select("message_id", "lista_destinatari")

# Add a column "#destinatari_per_email" with the number of recipients per email
result_df_dest = result_df_dest.withColumn(
    "#destinatari_per_email",
    F.size("lista_destinatari")
)

# Group by the number of recipients and count the number of emails
numero_dest = result_df_dest.groupBy("#destinatari_per_email").count().withColumnRenamed("count", "num_of_email_con_#destinatari")
numero_dest_sorted = numero_dest.orderBy(F.desc("num_of_email_con_#destinatari"))


# Filter the DataFrame to get message IDs with #destinatari_per_email > 10
message_ids_above_10_df = result_df_dest.filter(result_df_dest["#destinatari_per_email"] > 15)

# Collect the message IDs as a list
message_id_TODROP = message_ids_above_10_df.select("message_id").rdd.flatMap(lambda x: x).collect()




print(len(message_id_TODROP))




1  FILE caricato: email_60-79.csv
2  FILE caricato: email_0-19.csv
3  FILE caricato: email_20-39.csv
4  FILE caricato: email_80-99.csv
5  FILE caricato: email_40-59.csv
6  FILE caricato: email_100-119.csv
7  FILE caricato: email_120-139.csv
8  FILE caricato: email_180-199.csv
9  FILE caricato: email_160-179.csv
10  FILE caricato: email_140-159.csv
11  FILE caricato: email_220-239.csv
12  FILE caricato: email_240-259.csv
13  FILE caricato: email_200-219.csv
14  FILE caricato: email_260-279.csv
15  FILE caricato: email_280-299.csv
16  FILE caricato: email_300-319.csv
17  FILE caricato: email_340-359.csv
18  FILE caricato: email_360-379.csv
19  FILE caricato: email_320-339.csv
20  FILE caricato: email_400-419.csv
21  FILE caricato: email_380-399.csv
22  FILE caricato: email_440-459.csv
23  FILE caricato: email_420-439.csv
24  FILE caricato: email_480-500.csv
25  FILE caricato: email_460-479.csv
1603050


In [ ]:

# Filter the DataFrame to get message IDs with #destinatari_per_email > 25
message_ids_above_30_df = result_df_dest.filter(result_df_dest["#destinatari_per_email"] > 25)

# Collect the message IDs as a list
message_id_TODROP = message_ids_above_10_df.select("message_id").rdd.flatMap(lambda x: x).collect()

print(len(message_id_TODROP))


1154705


In [ ]:
# Vado a contare il numero di righe nel dataset che rimangono dopo la pulizia


# Converto la lista in un DataFrame per usarla nell'operazione di join
message_id_df = spark.createDataFrame([(message_id,) for message_id in message_id_TODROP], ['message_id'])

# Faccio la join per eliminare le righe con message_id presenti nella lista
df_email_spark_filtered = df_email_spark.join(message_id_df, on='message_id', how='left_anti')

# Conta il numero di righe rimaste
num_rows_remaining = df_email_spark_filtered.count()

# Stampa il risultato
print("Numero di righe prima:", df_email_spark.count())
print("Numero di righe rimaste in df_email_spark:", num_rows_remaining)

Numero di righe prima: 87296456
Numero di righe rimaste in df_email_spark: 43599878


In [ ]:
#Vado a calcolare le feature per ogni persona andando a leggere il relativo file contenente tutte le email dove compare come mittente o come destiantario

processato=0

#csv_file_path = '/content/gdrive/MyDrive/processati_email/chunked_userID'


try:

  for filename in os.listdir(csv_file_path):
      if filename.endswith(".csv"):

          # Extract the user_id from the filename by removing the ".csv" extension
          ID_persona = filename[:-4]

          # Read the CSV file into a DataFrame
          csv_file = os.path.join(csv_file_path, filename)
          df_email_completo = pd.read_csv(csv_file)

          # Siccome non l'ho fatto il precedenza, vado ad eliminare tutte quelle email che avevano un numero di destinatari eccessivamente alto!
          # per fare ciò in 2.1_PYSPARK:numeriche ho estratto tutti i memssage_id con numero di destinatari >25 e qui li vado a scartare
          df_email_completo = df_email_completo[~df_email_completo["message_id"].isin(message_id_TODROP)]


          processato= processato+1
          if processato%100==0 : print(processato)
          if ((ID_persona=='External') or  (ID_persona=='Consulenti') or  (ID_persona=='ND')or  (ID_persona=='Group')): continue


          # ///////////////////// pre-process the data for each user \\\\\\\\\\\\\\\\\\\\\\\\\\
          email_inviate_da_ID = df_email_completo[df_email_completo['source_from_header_address'] == ID_persona]
          email_ricevute_da_ID = df_email_completo[df_email_completo['destination_address'] == ID_persona]
          anagrafica_ID= df_anagrafici_excel[df_anagrafici_excel['CODICI RISPONDENTI'] == ID_persona]
          anagrafica_destinatari_di_ID = df_anagrafici_excel[df_anagrafici_excel['CODICI RISPONDENTI'].isin(email_inviate_da_ID['destination_address'].unique())]
          anagrafica_mittenti_di_ID = df_anagrafici_excel[df_anagrafici_excel['CODICI RISPONDENTI'].isin(email_ricevute_da_ID['source_from_header_address'].unique())]



          # ///////////////////////// info anagrafiche \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

          #calcolo la seniority in anni (anno corrente - anno assunzione)
          dataAssunzione = anagrafica_ID['Data prima assunzione'].values[0]
          seniorityCalcolata= (int(currentYear) - int( dataAssunzione[6:len(dataAssunzione)] )) if dataAssunzione !=0 else 0  ## if necessario perché nel dataset un valore era a 0
          df_features.loc[df_features['user_ID'] == ID_persona, 'seniority'] = seniorityCalcolata

          # prendo la Posizione(Addetto/Manager) di ogni persona
          addetto_manager_value = anagrafica_ID['Posizione'].values
          df_features.loc[df_features['user_ID'] == ID_persona, 'addettoVSmanager'] = addetto_manager_value[0] if len(addetto_manager_value)>0 else None

          # prendo la retribuzione (in scala da 1 a 100) di ogni persona
          retribuzione_value = df_retribuzione_excel.loc[df_retribuzione_excel['CODICI RISPONDENTI'] == ID_persona, 'SCALA 100'].values
          df_features.loc[df_features['user_ID'] == ID_persona, 'retribuzione'] = retribuzione_value[0] if len(retribuzione_value) > 0 else 0


          # prendo le performance di ogni persona, utilizzo la valutazione sintetica, cioè quella che prende in considerazioni più fattori
          # ho deciso di prendere questa perché stando al HR di CREDEM è quella più completa e considera nel complessivo come opera la persona
          # vado a trasformare i valori testuali/categorici in valori numerici secondo il seguente mapping:
          # ECCELLENTI = 7
          # RILEVANTI = 6
          # APPREZZABILI = 5
          # SODDISFACENTI = 4
          # ADEGUATI = 3
          # CARENTI = 2
          # INADEGUATI = 1
          # 0 = siccome indica la mancanza di informazioni e non che le performance sono 0, lo sostituisco con 'null'
          performance_value = anagrafica_ID['Val. 2021'].values
          df_features.loc[df_features['user_ID'] == ID_persona, 'performance'] = performance_value[0] if len(performance_value)>0 else None
          # Converti '0' in None, 0 = siccome indica la mancanza di informazioni e non che le performance sono 0, lo sostituisco con 'null'
          df_features['performance'] = df_features['performance'].replace(0, None)
          # Effettua la sostituzione dei valori testuali con i valori numerici
          df_features['performance'] = df_features['performance'].replace(mapping)



          # ///////////////////////// statistiche email  \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

          # prendo le email inviate da una persona, se l'email ha più destinatari conta come più email inviate
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_sent_email'] = len(email_inviate_da_ID)

          # prendo le email ricevute da una persona
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_received_email'] = len(email_ricevute_da_ID)

          # Numero di persone distinte con cui ha interagito
          df_features.loc[df_features['user_ID'] == ID_persona, '#persone_interazioni'] = len(set(email_inviate_da_ID['destination_address']) | set(email_ricevute_da_ID['source_from_header_address'])) # tolto perché ho filtrato via le email invate ase stesso: - 1  # Subtract 1 to exclude ID_persona itself

          # email inviate ad EXTERNAL
          df_features.loc[df_features['user_ID'] == ID_persona, 'sent_to_EXTERNAL'] = len(email_inviate_da_ID[email_inviate_da_ID['destination_address'] == 'External'])

          # email ricevute da EXTERNAL
          df_features.loc[df_features['user_ID'] == ID_persona, 'received_from_EXTERNAL'] = len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'] == 'External'])

          #email inviate a CONSULENTI
          df_features.loc[df_features['user_ID'] == ID_persona, 'sent_to_CONSULENTI'] = len(email_inviate_da_ID[email_inviate_da_ID['destination_address'] == 'Consulenti'])

          #email ricevute da CONSULENTI
          df_features.loc[df_features['user_ID'] == ID_persona, 'received_from_CONSULENTI'] = len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'] == 'Consulenti'])



          # //////////////////////////////////////////////////////// BUSINESS UNIT \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
          #ATTENZIONE: qui non sto calcolando quante email ha inviato/ricevute da quelle persone, ma quante persone distinte con quelle caratteristiche ha interagito

          # Preparazione per BU SUP ed INF
          destination_addresses = email_inviate_da_ID['destination_address'].unique()
          receiver_addressess= email_ricevute_da_ID['source_from_header_address'].unique()

          righe_di_anagrafica_dei_destinatari = anagrafica_destinatari_di_ID     # Filter the rows in df_anagrafici_excel for the given destination addresses: così ho tutta la riga e viene facile vedere la BU
          righe_di_anagrafica_dei_mittenti = anagrafica_mittenti_di_ID  # Filter the rows in df_anagrafici_excel for the given receiver_addressess: così ho tutta la riga e viene facile vedere la BU

          BU_sup_ID_persona = anagrafica_ID['UO sup'].values
          BU_inf_ID_persona = anagrafica_ID['Codice UO'].values

          # aggiungo nel df_feature anche queste due informazioni, così in un secondo momento sarà più facile recuperarle
          df_features.loc[df_features['user_ID'] == ID_persona, 'BU_sup'] = BU_sup_ID_persona[0] if len(BU_sup_ID_persona)>0 else None
          df_features.loc[df_features['user_ID'] == ID_persona, 'BU_inf'] = BU_inf_ID_persona[0] if len(BU_inf_ID_persona)>0 else None

          #persone della stessa BU sup a cui ha inviato email
          people_sent_SAME_bu_SUP =anagrafica_destinatari_di_ID[anagrafica_destinatari_di_ID['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_sent_SAME_bu_SUP'] = people_sent_SAME_bu_SUP.nunique()

          # numero di email inviate a persone della stessa BU sup
          email_sent_SAME_bu_SUP =len(email_inviate_da_ID[email_inviate_da_ID['destination_address'].isin(anagrafica_destinatari_di_ID[anagrafica_destinatari_di_ID['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_SAME_bu_SUP'] = email_sent_SAME_bu_SUP

          #persone della stessa BU inf a cui ha inviato delle email
          people_sent_SAME_bu_INF = anagrafica_destinatari_di_ID[anagrafica_destinatari_di_ID['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_sent_SAME_bu_INF'] = people_sent_SAME_bu_INF.nunique()

          # numero di email inviate a persone della stessa BU inf
          email_sent_SAME_bu_INF= len(email_inviate_da_ID[email_inviate_da_ID['destination_address'].isin(anagrafica_destinatari_di_ID[anagrafica_destinatari_di_ID['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_SAME_bu_INF'] = email_sent_SAME_bu_INF

          #persone con una BU sup diversa a cui ha invaito delle email
          people_sent_different_bu_SUP =  anagrafica_destinatari_di_ID[~anagrafica_destinatari_di_ID['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_sent_different_bu_SUP'] = people_sent_different_bu_SUP.nunique()

          # numero di email inviate a persone con diversa BU sup
          email_sent_different_bu_SUP= len(email_inviate_da_ID[email_inviate_da_ID['destination_address'].isin(anagrafica_destinatari_di_ID[~anagrafica_destinatari_di_ID['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_different_bu_SUP'] = email_sent_different_bu_SUP

          #email inviate a persone con una BU inf diversa
          people_sent_different_bu_INF= anagrafica_destinatari_di_ID[~anagrafica_destinatari_di_ID['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_sent_different_bu_INF'] = people_sent_different_bu_INF.nunique()

          # numero di email inviate a persone con diversa BU inf
          email_sent_different_bu_INF= len(email_inviate_da_ID[email_inviate_da_ID['destination_address'].isin(anagrafica_destinatari_di_ID[~anagrafica_destinatari_di_ID['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_different_bu_INF'] = email_sent_different_bu_INF

          # persone ricevute che hanno inviato email dalla stessa BU sup
          people_received_SAME_bu_SUP =righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_received_SAME_bu_SUP'] = people_received_SAME_bu_SUP.nunique()

          # email ricevute da persone della stessa BU sup
          email_received_SAME_bu_SUP= len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'].isin(righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_SAME_bu_SUP'] = email_received_SAME_bu_SUP

          #persone ricevute da persone della stess BU inf
          people_received_SAME_bu_INF= righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_received_SAME_bu_INF'] = people_received_SAME_bu_INF.nunique()

          # email ricevute da persone della stessa BU inf
          email_received_SAME_bu_INF=len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'].isin(righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_SAME_bu_INF'] = email_received_SAME_bu_INF

          #persone ricevute da persone con una BU sup diversa
          people_received_DIFFERENT_bu_SUP= righe_di_anagrafica_dei_mittenti[~righe_di_anagrafica_dei_mittenti['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_received_DIFFERENT_bu_SUP'] = people_received_DIFFERENT_bu_SUP.nunique()

          #email ricevute da persone con una BU sup diversa
          email_received_DIFFERENT_bu_SUP= len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'].isin(righe_di_anagrafica_dei_mittenti[~righe_di_anagrafica_dei_mittenti['UO sup'].isin(anagrafica_ID['UO sup'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_DIFFERENT_bu_SUP'] =  email_received_DIFFERENT_bu_SUP

          #email ricevute da persone con una BU inf diversa
          people_received_DIFFERENT_bu_INF = righe_di_anagrafica_dei_mittenti[~righe_di_anagrafica_dei_mittenti['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI']
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_received_DIFFERENT_bu_INF'] = people_received_DIFFERENT_bu_INF.nunique()

          #email ricevute da persone con una BU sup diversa
          email_received_DIFFERENT_bu_INF = len(email_ricevute_da_ID[email_ricevute_da_ID['source_from_header_address'].isin(righe_di_anagrafica_dei_mittenti[~righe_di_anagrafica_dei_mittenti['Codice UO'].isin(anagrafica_ID['Codice UO'])]['CODICI RISPONDENTI'])])
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_DIFFERENT_bu_INF'] = email_received_DIFFERENT_bu_INF

          # sent+received same BU sup
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_total_SAME_bu_SUP'] = len(set(people_sent_SAME_bu_SUP) | set(people_received_SAME_bu_SUP))
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_SAME_bu_SUP'] = int(email_sent_SAME_bu_SUP) + int(email_received_SAME_bu_SUP)

          # sent+received same BU inf
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_total_SAME_bu_INF'] = len(set(people_sent_SAME_bu_INF) | set(people_received_SAME_bu_INF))
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_SAME_bu_INF'] = int(email_sent_SAME_bu_INF) + int(email_received_SAME_bu_INF)

          # sent+received different BU sup
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_total_DIFFERENT_bu_SUP'] = len(set(people_sent_different_bu_SUP) | set(people_received_DIFFERENT_bu_SUP))
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_DIFFERENT_bu_SUP'] = int(email_sent_different_bu_SUP) + int(email_received_DIFFERENT_bu_SUP)

          # sent+received different BU inf
          df_features.loc[df_features['user_ID'] == ID_persona, 'people_total_DIFFERENT_bu_INF'] = len(set(people_sent_different_bu_INF) | set(people_received_DIFFERENT_bu_INF))
          df_features.loc[df_features['user_ID'] == ID_persona, 'total_DIFFERENT_bu_INF'] = int(email_received_DIFFERENT_bu_INF) + int(email_sent_different_bu_INF)


          # Calculate email interactions within ADDETTO and MANAGER
          persone_sent_to_ADDETTO = righe_di_anagrafica_dei_destinatari[righe_di_anagrafica_dei_destinatari['Posizione'] == 'Addetto']['CODICI RISPONDENTI'].values
          persone_sent_to_MANAGER = righe_di_anagrafica_dei_destinatari[righe_di_anagrafica_dei_destinatari['Posizione'] == 'Manager']['CODICI RISPONDENTI'].values
          persone_received_from_ADDETTO = righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['Posizione'] == 'Addetto']['CODICI RISPONDENTI'].values
          persone_received_from_MANAGER = righe_di_anagrafica_dei_mittenti[righe_di_anagrafica_dei_mittenti['Posizione'] == 'Manager']['CODICI RISPONDENTI'].values

          email_inviate_ADDETTO = df_email_completo[(df_email_completo['source_from_header_address'] == ID_persona) & (df_email_completo['destination_address'].isin(persone_sent_to_ADDETTO))]
          email_inviate_MANAGER = df_email_completo[(df_email_completo['source_from_header_address'] == ID_persona) & (df_email_completo['destination_address'].isin(persone_sent_to_MANAGER))]
          email_ricevute_ADDETTO = df_email_completo[(df_email_completo['destination_address'] == ID_persona) & (df_email_completo['source_from_header_address'].isin(persone_received_from_ADDETTO))]
          email_ricevute_MANAGER = df_email_completo[(df_email_completo['destination_address'] == ID_persona) & (df_email_completo['source_from_header_address'].isin(persone_received_from_MANAGER))]

          # Update the DataFrame with the calculated values
          df_features.loc[df_features['user_ID'] == ID_persona, 'persone_sent_to_ADDETTO'] = len(persone_sent_to_ADDETTO)
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_to_ADDETTO'] = len(email_inviate_ADDETTO)
          df_features.loc[df_features['user_ID'] == ID_persona, 'persone_sent_to_MANAGER'] = len(persone_sent_to_MANAGER)
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_sent_to_MANAGER'] = len(email_inviate_MANAGER)
          df_features.loc[df_features['user_ID'] == ID_persona, 'persone_received_from_ADDETTO'] = len(persone_received_from_ADDETTO)
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_from_ADDETTO'] = len(email_ricevute_ADDETTO)
          df_features.loc[df_features['user_ID'] == ID_persona, 'persone_received_from_MANAGER'] = len(persone_received_from_MANAGER)
          df_features.loc[df_features['user_ID'] == ID_persona, 'email_received_from_MANAGER'] = len(email_ricevute_MANAGER)

except KeyError as e:
    print(f"Error processing file: {filename}. KeyError: {e}")

100
200


<ipython-input-10-d1252639eedc>:16: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_email_completo = pd.read_csv(csv_file)


300
400
500
600
700
800
900
1000
1100
1200
1300


<ipython-input-10-d1252639eedc>:16: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_email_completo = pd.read_csv(csv_file)


1400
1500
1600
1700
1800
1900
2000


In [ ]:

#GUARDA LE STATISTICHE DELLE FEATURE PER CAPIRE SE HANNO SENSO
print(df_features.describe().to_string())
print()
print()

print(df_features.info())
print()
print()

print(len(df_features))
print()
print()

print(df_features.head(8).to_string())
print()
print()


       performance    seniority  retribuzione  total_sent_email  total_received_email  #persone_interazioni  sent_to_EXTERNAL  received_from_EXTERNAL  sent_to_CONSULENTI  received_from_CONSULENTI  people_sent_SAME_bu_SUP  email_sent_SAME_bu_SUP  people_sent_SAME_bu_INF  email_sent_SAME_bu_INF  email_sent_different_bu_SUP  people_sent_different_bu_SUP  people_sent_different_bu_INF  email_sent_different_bu_INF  people_received_SAME_bu_SUP  email_received_SAME_bu_SUP  people_received_SAME_bu_INF  email_received_SAME_bu_INF  people_received_DIFFERENT_bu_SUP  email_received_DIFFERENT_bu_SUP  email_received_DIFFERENT_bu_INF  people_received_DIFFERENT_bu_INF  total_SAME_bu_SUP  people_total_SAME_bu_SUP  total_SAME_bu_INF  people_total_SAME_bu_INF  total_DIFFERENT_bu_SUP  people_total_DIFFERENT_bu_SUP  total_DIFFERENT_bu_INF  people_total_DIFFERENT_bu_INF  persone_sent_to_ADDETTO  email_sent_to_ADDETTO  persone_sent_to_MANAGER  email_sent_to_MANAGER  persone_received_from_ADDETTO  email_receiv

In [ ]:
csv_file_path = '/content/gdrive/MyDrive/processati_email/df_features2.csv'  # Specify the file path where you want to save the CSV
df_features.to_csv(csv_file_path, index=False)

print("DataFrame df_email_completo saved to CSV successfully!")

DataFrame df_email_completo saved to CSV successfully!


In [ ]:
print(df_features[ (df_features['total_sent_email']!=0) & (df_features['total_received_email']!=0) ].describe().to_string())

       performance    seniority  retribuzione  total_sent_email  total_received_email  #persone_interazioni  sent_to_EXTERNAL  received_from_EXTERNAL  sent_to_CONSULENTI  received_from_CONSULENTI  people_sent_SAME_bu_SUP  email_sent_SAME_bu_SUP  people_sent_SAME_bu_INF  email_sent_SAME_bu_INF  email_sent_different_bu_SUP  people_sent_different_bu_SUP  people_sent_different_bu_INF  email_sent_different_bu_INF  people_received_SAME_bu_SUP  email_received_SAME_bu_SUP  people_received_SAME_bu_INF  email_received_SAME_bu_INF  people_received_DIFFERENT_bu_SUP  email_received_DIFFERENT_bu_SUP  email_received_DIFFERENT_bu_INF  people_received_DIFFERENT_bu_INF  total_SAME_bu_SUP  people_total_SAME_bu_SUP  total_SAME_bu_INF  people_total_SAME_bu_INF  total_DIFFERENT_bu_SUP  people_total_DIFFERENT_bu_SUP  total_DIFFERENT_bu_INF  people_total_DIFFERENT_bu_INF  persone_sent_to_ADDETTO  email_sent_to_ADDETTO  persone_sent_to_MANAGER  email_sent_to_MANAGER  persone_received_from_ADDETTO  email_receiv